In [ ]:
"""
Author: Jungmyung Lee

3D Marker-Based Skeleton Reconstruction (Static Frame Visualization)
--------------------------------------------------------------------
This script loads a Qualisys-style motion-capture .mat file and reconstructs
a biomechanical stick-figure model from labeled marker trajectories.

Unlike the full animation pipeline, this script focuses on:
- examining marker validity,
- selecting the most recent valid 3D coordinates per marker,
- building a static stick-figure representation of the subject,
- verifying marker naming consistency and anatomical mapping.

Main processing steps:
1) Load labeled marker trajectories from a .mat file
2) Apply a robust fallback parser (supports multiple QTM export formats)
3) Automatically match anatomical markers through prefix-based name search
4) Reconstruct major body segments (pelvis, torso, limbs, feet)
5) Select either:
      • a specific frame index, or
      • the last valid sample for each marker (default behavior)
6) Render a 3D static stick-figure visualization with equalized axes

This script is primarily designed for:
- quality inspection of raw mocap data,
- validating marker labeling conventions,
- checking reconstruction accuracy before generating full animations.
"""


import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import scipy.io as sio
try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# ===== 1) Upload .mat file (Colab) or set path (local) =====
if IN_COLAB:
    uploaded = files.upload()
    path = list(uploaded.keys())[0]
else:
    path = 'cutting.mat'  # change if running locally
print('Loaded file:', path)

# ===== 2) Load and flexibly parse Trajectories → Labeled =====
data = sio.loadmat(path, squeeze_me=True, struct_as_record=False)
user_keys = [k for k in data.keys() if not (k.startswith('__') and k.endswith('__'))]
top_key = user_keys[0]
S = data[top_key]

def get_labeled_block(S):
    # Case A: MATLAB mat-struct
    if hasattr(S, 'Trajectories'):
        T = S.Trajectories
        L = T.Labeled if hasattr(T, 'Labeled') else None
        if L is not None and hasattr(L, 'Data'):
            labels = list(L.Labels) if isinstance(L.Labels, (list, tuple, np.ndarray)) else [L.Labels]
            arr = np.array(L.Data)
            return [str(x) for x in labels], arr
    # Case B: numpy structured array (dtype with 'Labeled')
    if isinstance(S, np.void) or isinstance(S, np.ndarray):
        try:
            T = S['Trajectories'][()] if isinstance(S, np.void) else S['Trajectories']
        except Exception:
            T = S
        if isinstance(T, np.ndarray) and T.dtype.names and 'Labeled' in T.dtype.names:
            L = T['Labeled'][0,0]
            labels_raw = L['Labels'][0,0]
            data_raw = L['Data'][0,0]
            if isinstance(labels_raw, np.ndarray):
                labels = [str(x.item() if hasattr(x, 'item') else x) for x in labels_raw.ravel()]
            else:
                labels = [str(labels_raw)]
            arr = np.array(data_raw)
            return labels, arr
    raise RuntimeError('Could not find Trajectories → Labeled → Data in this file.')

labels, arr = get_labeled_block(S)
print(f'Markers: {len(labels)} | Data shape: {arr.shape} (markers, 4, frames)')
frames = arr.shape[2]

# ===== 3) Marker-name helper (handles slight name variants by prefix match) =====
def find_idx(name_candidates):
    for cand in name_candidates:
        for i, lab in enumerate(labels):
            if str(lab).upper().startswith(cand.upper()):
                return i
    return None

# Common markers in your file
M = {
    'STRN': find_idx(['STRN']), 'XPHOID': find_idx(['XPHOID','XPHO']),
    'C7': find_idx(['C7']), 'T10': find_idx(['T10']),
    'LACR': find_idx(['LACR']), 'RACR': find_idx(['RACR']),
    'LRSP': find_idx(['LRSP']), 'RRSP': find_idx(['RRSP']),
    'LUSP': find_idx(['LUSP']), 'RUSP': find_idx(['RUSP']),
    'LELB': find_idx(['LELB']), 'RELB': find_idx(['RELB']),
    'LUPPERARM': find_idx(['LUPPERARM']), 'RUPPERARM': find_idx(['RUPPERARM']),
    'LFOREARM': find_idx(['LFOREARM']), 'RFOREARM': find_idx(['RFOREARM']),
    'LHAND': find_idx(['LHAND']), 'RHAND': find_idx(['RHAND']),
    'LASIS': find_idx(['LASIS']), 'RASIS': find_idx(['RASIS']),
    'LPSIS': find_idx(['LPSIS']), 'RPSIS': find_idx(['RPSIS']),
    'LTHIGH': find_idx(['LTHIGH']), 'RTHIGH': find_idx(['RTHIGH']),
    'LEPI': find_idx(['LEPI','LEPI_L']), 'REPI': find_idx(['REPI','REPI_L']),
    'LSHANK': find_idx(['LSHANK']), 'RSHANK': find_idx(['RSHANK']),
    'LMAL': find_idx(['LMAL','LMAL_L']), 'RMAL': find_idx(['RMAL','RMAL_L']),
    'LHEEL': find_idx(['LHEEL']), 'RHEEL': find_idx(['RHEEL']),
    'L5META': find_idx(['L5THMETA','L5META']), 'R5META': find_idx(['R5THMETA','R5META']),
    'LTOE': find_idx(['LTOE']), 'RTOE': find_idx(['RTOE']),
    'LIC': find_idx(['LIC']), 'RIC': find_idx(['RIC']),
}

# ===== 4) Skeleton edges =====
edges = [
    ('LASIS','RASIS'), ('LPSIS','RPSIS'), ('LASIS','LPSIS'), ('RASIS','RPSIS'),
    ('STRN','XPHOID'), ('STRN','C7'), ('C7','T10'), ('LACR','RACR'),
    ('STRN','LACR'), ('STRN','RACR'),
    ('RACR','RUPPERARM'), ('RUPPERARM','RELB'), ('RELB','RFOREARM'), ('RFOREARM','RHAND'),
    ('LACR','LUPPERARM'), ('LUPPERARM','LELB'), ('LELB','LFOREARM'), ('LFOREARM','LHAND'),
    ('RASIS','RTHIGH'), ('RTHIGH','REPI'), ('REPI','RSHANK'), ('RSHANK','RMAL'),
    ('RMAL','RHEEL'), ('RMAL','R5META'), ('RMAL','RTOE'),
    ('LASIS','LTHIGH'), ('LTHIGH','LEPI'), ('LEPI','LSHANK'), ('LSHANK','LMAL'),
    ('LMAL','LHEEL'), ('LMAL','L5META'), ('LMAL','LTOE'),
]

# ===== 5) Choose a frame; -1 = last valid per marker =====
frame_index = -1
residual_threshold = 2.0

def marker_xyz(i, f):
    if i is None:
        return None
    xyzr = arr[i, :, :]
    x, y, z, r = xyzr[0], xyzr[1], xyzr[2], xyzr[3]
    if frame_index == -1:
        mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z)
        if residual_threshold is not None:
            mask &= np.isfinite(r) & (r <= residual_threshold)
        if not np.any(mask):
            return None
        k = np.where(mask)[0][-1]
        return x[k], y[k], z[k]
    else:
        f = int(np.clip(f, 0, frames-1))
        ok = np.isfinite(x[f]) and np.isfinite(y[f]) and np.isfinite(z[f])
        if residual_threshold is not None:
            ok = ok and np.isfinite(r[f]) and (r[f] <= residual_threshold)
        return (x[f], y[f], z[f]) if ok else None

def set_equal_3d(ax):
    xlim = ax.get_xlim3d(); ylim = ax.get_ylim3d(); zlim = ax.get_zlim3d()
    xmid = np.mean(xlim); ymid = np.mean(ylim); zmid = np.mean(zlim)
    max_range = max(xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0])
    r = max_range / 2.0
    ax.set_xlim3d([xmid - r, xmid + r])
    ax.set_ylim3d([ymid - r, ymid + r])
    ax.set_zlim3d([zmid - r, zmid + r])

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

for a_key, b_key in edges:
    ia, ib = M.get(a_key), M.get(b_key)
    Pa = marker_xyz(ia, frame_index)
    Pb = marker_xyz(ib, frame_index)
    if (Pa is None) or (Pb is None):
        continue
    xa, ya, za = Pa; xb, yb, zb = Pb
    ax.plot([xa, xb], [ya, yb], [za, zb], linewidth=2)

for i, name in enumerate(labels):
    P = marker_xyz(i, frame_index)
    if P is None:
        continue
    ax.scatter(P[0], P[1], P[2], s=14)

ax.set_title('Stick Figure + Markers')
ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
set_equal_3d(ax)
plt.show()